In [1]:
import pandas as pd
import numpy as np
import time
import glob
import os, platform
from datetime import datetime

In [2]:
if platform.system()=='Darwin':
    path = '/Users/phil/Google Drive/projects/nfl/data'
elif os.environ.get("USERNAME")=='phil':
    path = 'C:/Users/phil/Google Drive/nfl/data'
elif os.environ.get("USERNAME")=='lyncp010':
    path = 'C:/Users/lyncp010/projects/nfl/data'
print path

C:/Users/lyncp010/projects/nfl/data


In [3]:
# list of dataframes
gmin = 'gameInfo'
inst = 'injuryStatus'
psdr = 'passDirections'
snct = 'snapCounts'
strt = 'starters'
sdef = 'statsDefense'
skck = 'statsKicking'
soff = 'statsOffense'
srtn = 'statsReturns'
stm  = 'statsTeam'
tmrs = 'teamRoster'

dfl = [gmin, inst, psdr, snct, strt, sdef, skck, soff, srtn, stm, tmrs]

In [4]:
dfIn = {
    'gameInfo'      :['season','week','bsID'                   ],
    'injuryStatus'  :['season','week',       'team','player_id'],
    'passDirections':['season','week','bsID','team','player_id'],
    'snapCounts'    :['season','week','bsID','team','player_id'],
    'starters'      :['season','week','bsID','team','player_id'],
    'statsDefense'  :['season','week','bsID','team','player_id'],
    'statsKicking'  :['season','week','bsID','team','player_id'],
    'statsOffense'  :['season','week','bsID','team','player_id'],
    'statsReturns'  :['season','week','bsID','team','player_id'],
    'statsTeam'     :['season','week','bsID','team'            ],
    'teamRoster'    :['season',              'team','player_id'],
}

In [8]:
# import csvs into dataframes
d = {}
for key in dfIn:
    print key
    files = glob.glob(os.path.join(path, 'all', '{}*.csv'.format(key)))
    d[key] = pd.concat((pd.read_csv(f, low_memory=False) for f in files), ignore_index=True)
    d[key].set_index(dfIn[key], inplace=True)
    d[key].sortlevel(inplace=True)

gameInfo
statsTeam
statsDefense
snapCounts
passDirections
statsKicking
teamRoster
starters
injuryStatus
statsOffense
statsReturns


In [9]:
# print df index + columns
for df in dfl:
    print df
    print '-'*len(df)
    for i in d[df].index.names: print '*', i
    for i in d[df].columns: print i
    print '\n'

gameInfo
--------
* season
* week
* bsID
date
weekday
startTime
home
away
winner
homeScore
awayScore
line
overUnder
roof
surface
temp
relHumidity
windChill
windMPH


injuryStatus
------------
* season
* week
* team
* player_id
didNotPlay
playerID
status


passDirections
--------------
* season
* week
* bsID
* team
* player_id
rec_catches_d
rec_catches_dl
rec_catches_dm
rec_catches_dr
rec_catches_s
rec_catches_sl
rec_catches_sm
rec_catches_sr
rec_targets_d
rec_targets_dl
rec_targets_dm
rec_targets_dr
rec_targets_s
rec_targets_sl
rec_targets_sm
rec_targets_sr
rec_td_dl
rec_td_dm
rec_td_dr
rec_td_sl
rec_td_sm
rec_td_sr
rec_yds_dl
rec_yds_dm
rec_yds_dr
rec_yds_sl
rec_yds_sm
rec_yds_sr
rec_catches_no_dir
rec_targets_no_dir
rec_td_no_dir
rec_yds_no_dir


snapCounts
----------
* season
* week
* bsID
* team
* player_id
position
offSnaps
offSnapsPct
defSnaps
defSnapsPct
stSnaps
stSnapsPct


starters
--------
* season
* week
* bsID
* team
* player_id
playerName
position
home
offense


statsDefen

# fixing data

In [10]:
idx = pd.IndexSlice

In [11]:
# start with offense stats
df = d[soff].copy().reset_index()

In [12]:
# add home indicator
df['home'] = (df['bsID'].str[-3:]==df['team']).astype(int)

In [13]:
# change new team abbreviations to old/consistent ones (i.e. lar > ram)
new_team_name_dict = {
    'ari':'crd',
    'bal':'rav',
    'hou':'htx',
    'ind':'clt',
    'lar':'ram',
    'stl':'ram',
    'oak':'rai',
    'ten':'oti'
}

df['team'].replace(new_team_name_dict, inplace=True)

In [14]:
# add opponent
homeAwayOpp = {True:'away', False:'home'}
for i, r in df.iterrows():
    seas, week, bsid, team, plid = r['season'], r['week'], r['bsID'], r['team'], r['player_id']
    foo = d[gmin].loc[idx[[seas],[week],[bsid]],]
    df.loc[i, 'opponent'] = foo[homeAwayOpp[foo['home'][0]==team]][0]

In [15]:
# merge in player positions
df = pd.merge(df.reset_index(),
              d[tmrs].reset_index()[['season', 'player_id', 'position']],
              how='left',
              on = ['season', 'player_id']
             )
df['position'] = df['position'].str.upper()

## missing positions

In [26]:
miss_list = df[df['position'].isnull()]['player_id'].unique()
print 'Missing player positions - {}'.format(len(miss_list))
miss_list

Missing player positions - 0


array([], dtype=object)

#### load file of missing players

In [17]:
# load missing player dictionary
miss_dict = np.load(path + '/all/missing_player_positions_dict.npy').item()

In [25]:
# fill in missing positions
df['position'].fillna(df['player_id'].map(miss_dict), inplace=True)

#### otherwise get positions from other files / scraping

In [28]:
# miss_dict = {}

In [22]:
# fill players with missing positions with their most documented position in stats offense df
miss_list = df[df['position'].isnull()]['player_id'].unique()
for pid in miss_list:
    foo = df[df['player_id']==pid]['position'].value_counts()
    if len(foo)>0:
        if pid not in miss_dict: miss_dict[pid] = foo.argmax()
        
df['position'].fillna(df['player_id'].map(miss_dict), inplace=True)

In [23]:
# fill players with missing positions with their most documented position in team roster df
miss_list = df[df['position'].isnull()]['player_id'].unique()
for pid in miss_list:
    try:
        foo = d[tmrs].loc[idx[:,:,pid],]['position'].value_counts()
        if len(foo) > 0:
            if pid not in miss_dict: miss_dict[pid] = foo.argmax()
    except:pass
        
df['position'].fillna(df['player_id'].map(miss_dict), inplace=True)

In [24]:
# fill players with missing positions with their most documented position in snap counts df
miss_list = df[df['position'].isnull()]['player_id'].unique()
for pid in miss_list:
    try:
        foo = d[snct].loc[idx[:,:,:,:,pid],]['position'].value_counts()
        if len(foo) > 0:
            if pid not in miss_dict: miss_dict[pid] = foo.argmax()
    except: pass
        
df['position'].fillna(df['player_id'].map(miss_dict), inplace=True)

#### scrape positions of everyone else

In [ ]:
import bs4
import urllib2
import re

In [ ]:
def scrape_player_id_position(pid):
    url = 'http://www.pro-football-reference.com/players/{}/{}.htm'.format(pid[0], pid)
    soup = bs4.BeautifulSoup(urllib2.urlopen(url).read(), "lxml")
    for x in soup.findAll(['p']):
        if 'Position' in x.text:
            match = re.search(r"Position:\s(.*)", x.text)
            if match:
                return match.group(1)
                #missing_players_dict[pid] = match.group(1)
                #print match.group(1)
            else: return np.nan

In [ ]:
# fill players with missing positions with their position from pfr player page
miss_list = df[df['position'].isnull()]['player_id'].unique()
n = len(miss_list)
for i, pid in enumerate(miss_list):
    print '\r{} ({}/{})'.format(pid, i+1,n),
    miss_dict[pid] = scrape_player_id_position(pid)
print '\rDone!'

In [ ]:
for pid in miss_list:
    if pid in miss_dict:
        if miss_dict[pid] == None:
            miss_dict.pop(pid, None)

In [ ]:
# adding a few last players...
miss_dict['DearJa00'] = 'C'
miss_dict['TupaTo20'] = 'P'
miss_dict['BartMi00'] = 'TE'
miss_dict['MetcEr00'] = 'RB'
miss_dict['hentrcra01'] = 'P'
miss_dict['rouentom01'] = 'P'
miss_dict['knorrmic01'] = 'P'
miss_dict['carnejoh01'] = 'K'
miss_dict['wilkijef01'] = 'K'
miss_dict['edingpau01'] = 'K'
miss_dict['brownkri01'] = 'K'
miss_dict['elamxjas01'] = 'K'

In [29]:
# update df
df['position'].fillna(df['player_id'].map(miss_dict), inplace=True)

In [30]:
# update Terell Pryor's position from QB to WR
for i, r in df[(df['player_id']=='PryoTe00') & (df['season']>=2015)].iterrows():
    df.loc[i,'position'] = 'WR'

In [31]:
# save dictionary of missing players
np.save(path + '/all/missing_player_positions_dict.npy', miss_dict) 

# list positions

In [32]:
df['position'] = df['position'].str.upper()

In [33]:
df['position'].unique()

array(['RB', 'TE', 'QB', 'WR', 'FB', 'RT', 'P', 'SE', 'FL', 'LG', 'LCB',
       'K', 'FS', 'LB', 'MLB', 'C', 'RCB', 'DT', 'DB', 'RG', 'NT', 'LT',
       'SS', 'CB', 'LS', u'DE', 'S', 'LOLB', 'T', 'HB', 'DL'], dtype=object)

In [34]:
# convert fullback to runningback
df['position'].replace({'FB':'RB'}, inplace=True)

#### calculate fantasy points

In [35]:
# add indicator for fantasy yard bonuses
df['passYdsOver300'] = np.where(df['passYds']>=300, 1, 0)
df['rushYdsOver100'] = np.where(df['rushYds']>=100, 1, 0)
df['recYdsOver100']  = np.where(df['recYds']>=100, 1, 0)

In [36]:
# standard offense ff points calculator
ffPtsDict = {
    'passTds':        ( 4.0),
    'passYds':        ( 0.04),
    'passYdsOver300': ( 0.0),
    'passInt':        (-1.0),
    'rushYds':        ( 0.1),
    'rushTd':         ( 6.0),
    'rushYdsOver100': ( 0.0),
    'fumbles':        (-1.0),
    'recYds':         ( 0.1),
    'rec':            ( 0.0),
    'recTds':         ( 6.0),
    'recYdsOver100':  ( 0.0)
}

In [37]:
# calculate fantasy points
B = np.array([ffPtsDict[stat] for stat in sorted(ffPtsDict.keys())])
X = df[sorted(ffPtsDict.keys())].as_matrix()
df['ff_points_total'] = np.dot(X, B)

In [41]:
print df['ff_points_total'].min()
print df['ff_points_total'].max()

-3.6
55.4


In [43]:
# show line with max fantas points
df.loc[df['ff_points_total'].argmax()]

index                      8200
season                     2003
week                         14
bsID               200312070den
team                        den
player_id              PortCl00
passAtt                       0
passCmp                       0
passYds                       0
passTds                       0
passLong                      0
passRating                  NaN
passSacked                  NaN
passSackedYds               NaN
passInt                       0
targets                       3
rec                           2
recYds                       36
recTds                        0
recLong                      30
rushAtt                      22
rushYds                     218
rushTd                        5
rushLong                     59
fumbles                       0
fumblesLost                 NaN
home                          1
opponent                    kan
position                     RB
passYdsOver300                0
rushYdsOver100                1
recYdsOv

#### create dataframe with just offense positions

In [45]:
df.set_index('index', inplace=True)

In [46]:
# offense positions
offense_positions = ['QB', 'RB', 'TE', 'WR']

In [47]:
df_off = df.copy()

In [49]:
cols = ['season', 'week', 'bsID', 'team', 'player_id', 'position', 'ff_points_total', 'home', 'opponent']
for c in df_off.columns:
    if c not in cols:
        cols.append(c)
df_off = df_off[cols]

In [50]:
df_off = df_off[df_off['position'].isin(offense_positions)]

In [51]:
path

'C:/Users/lyncp010/projects/nfl/data'

In [53]:
df_off.to_csv(path + '/all/offense_player_ff_points_s02w01_s16w21.csv', index=False)

# scrape coaches

#### sports ref

In [ ]:
from sportsref import nfl

In [ ]:
coaches_df = pd.DataFrame()

In [ ]:
# scrape coach id for each season / team
for s in range(2002, 2016+1):
    for i, tm_nm in enumerate(nfl.teams.list_teams(s)):
        print '\rSeason {} - Team {}'.format(s, i+1),
        tm = nfl.Team(tm_nm)
        try:
            coach_array = tm.head_coaches_by_game(s)
            coach_df = pd.DataFrame(data = coach_array,
                                    columns = ['coach'],
                                    index=[[2016]*len(coach_array), [tm_nm]*len(coach_array), range(1, len(coach_array)+1)]
                                   )
            coach_df.index.names = ['seaon', 'team', 'week']
            coaches_df = pd.concat([coaches_df, coach_df])
        except:
            print 'BAD {} - {}'.format(s, tm_nm)
            pass

In [ ]:
from pyquery import PyQuery as pq

In [ ]:
coach_names = {}

In [ ]:
# get coache names from their coach id
all_coaches = coaches_df['coach'].unique()
for i, c in enumerate(all_coaches):
    print '\rCoach {} / {}'.format(i+1, len(all_coaches)),
    doc = pq('http://www.pro-football-reference.com/coaches/{}.htm'.format(c))
    try:
        coach_names[c] = doc('div#meta h1:first').text()
    except:
        print 'BAD {}'.format(c)
        coach_names[c] = ''

In [ ]:
coaches_df['coach_name'] = coaches_df['coach'].map(coach_names)

In [ ]:
coaches_df = coaches_df.rename(columns={'coach':'coach_id'})

In [ ]:
coaches_df.to_csv(path + '/all/coaches_s2002w01_s2016w21.csv')#, index=False)